## Financial Theories: Context for éclat

Purpose: concise, directly applicable financial-theory guidance for éclat's Underwriting, Sanction, and Verification agents.

### 1. Credit Risk Fundamentals (PD, LGD, EAD, Expected Loss)

Credit risk is decomposed into Probability of Default (PD), Loss Given Default (LGD), and Exposure at Default (EAD). Expected Loss = PD × LGD × EAD; capital and pricing decisions rely on these components.

Relevance to éclat: The Underwriting Agent must consume PD/LGD/EAD signals (internal APIs or calculated proxies) and present explainable constraints when decisions are adverse. Translating these components into human-readable, actionable reasons reduces user confusion and preserves dignity.

Design implications:
- Surface which component primarily drove a decline (e.g., PD vs LGD) and the actionable input (missing income proof, high utilization).
- Keep numerical details optional; present a concise natural-language explanation plus a next step.

_________________________________________________________________________________________________
### 2. Credit Scoring Models & Explainability

Traditional scorecards (logistic regression) provide transparent risk weights; modern ML (tree ensembles, neural nets) provide accuracy gains but require explainability (SHAP, LIME) for regulatory and UX reasons.

Relevance to éclat: Use hybrid approach — robust, validated scorecards where possible; when ML models are used, compute local explanations (SHAP) and convert attributions into symbolic rules the Underwriting Agent can narrate.

Design implications:
- Implement a fallback interpretable model (scorecard) for human-facing explanations when ML explanations are ambiguous.
- Log model inputs and explanation artifacts into the audit trail (immutable artifact) for reviews and appeals.

_________________________________________________________________________________________________
### 3. Adverse Selection & Moral Hazard

Asymmetric information problems; applicants have private information (adverse selection) and may change behavior after contracting (moral hazard) — drive cautious underwriting and verification practices.

Relevance to éclat: The Verification Agent reduces adverse selection by validating identity and documents (fuzzy matching, libpostal). The system should treat data-quality signals (mismatched address, OCR errors) as explainable contributors, not automatic rejects.

Design implications:
- Use graded verification outcomes (verified / likely / unverified) and convey this gradation to the Master Agent rather than binary pass/fail signals.
- Where possible, require minimal yet reliable proofs to reduce friction (tiered verification).

_________________________________________________________________________________________________
### 4. Risk-Based Pricing & Sanctioning

Summary: Risk-based pricing aligns price/offer with assessed risk; sanction letters are both an operational artifact and a behavioral device (previewing offer can increase commitment).

Relevance to éclat: The Sanction Agent should present sanction previews responsibly — indicate conditions, contingencies, and probability bands rather than guaranteed disbursement when upstream checks remain pending.

Design implications:
- When showing sanction previews, include explicit disclaimers and conditional language, and log the provenance (which checks were used).
- Use preview as a momentum-preserving artefact but avoid definitive language until finalization.

---
### 5. Regulatory Compliance & Model Governance

Summary: Financial services are regulated; model governance requires validation, explainability, audit trails, versioning, and the ability to reproduce decisions for review (e.g., RBI guidelines, Basel documents).

Relevance to éclat: Every automated decision or explanation must be reproducible and traceable. The architecture's artifact writer (`artifacts/state_schema.json`, explanation logs, template commit hashes) supports governance and audits.

Design implications:
- Persist model versions, input snapshots, and explanation artifacts alongside human-readable narratives for appeals and audits.
- Ensure sanction-letter templates and their commits are signed/versioned (git + hashing) as part of the chain-of-custody.

---
### 6. Operational Metrics: What to monitor

Key metrics to evaluate financial impact and safety:
- Accuracy & calibration of PD estimates (Brier, calibration curves).
- False positive / false negative rates per demographic subgroup (fairness).
- Time-to-decision, escalation rates, and manual review burden.

Relevance to éclat: Monitor both performance and behavioral metrics (drop-off, NPS) to ensure the orchestration increases conversions without introducing unfairness or regulatory risk.

---
### 7. Minimal mapping: theory → éclat components

- PD/LGD/EAD → Underwriting Agent inputs and explainable outputs (SHAP → symbolic narrative).
- Verification gradations → Verification Agent; Master Agent uses gradation to adapt pacing.
- Risk-based pricing → Sanction Agent; preview conditional offers with provenance.
- Governance & audit → artifact writer, template versioning, explanation logs for appeals.

---
### 8. Selected references (project‑relevant)

- Thomas, L. C., Crook, J. N., & Edelman, D. B. (2017). *Credit Scoring and Its Applications*.
- Basel Committee on Banking Supervision. (2006). *International Convergence of Capital Measurement and Capital Standards (Basel II)*.
- Lundberg, S. M., & Lee, S.-I. (2017). A unified approach to interpreting model predictions (SHAP). *arXiv:1705.07874*.
- Hand, D. J., & Henley, W. E. (1997). Statistical classification methods in consumer credit scoring: a review. *Journal of the Royal Statistical Society*.

---
### 9. Calibration, Backtesting & Stress Testing

Calibration ensures PD estimates correspond to observed frequencies (calibration curves, Brier score). Backtesting (out-of-time validation) detects model drift; stress testing evaluates performance under adverse macro scenarios.

Relevance to éclat: schedule automated backtests and calibration checks and record results as artifacts. Trigger retraining or human review when calibration degrades beyond pre-defined thresholds.

Design implications:
- Store time-stamped snapshots of model inputs/outputs to enable reproducible backtests.
- Define thresholds for acceptable calibration drift and automated alerts for triage.

---
### 10. Portfolio Effects & Concentration Risk

Individual credit decisions aggregate into portfolio exposures. Concentration in sectors, geographies, or product types raises systemic risk even if individual PDs are low.

Relevance to éclat: the Master Agent should be able to request portfolio-aware constraints (e.g., throttle offers into a given segment) and the Evaluation notebook should compute concentration metrics periodically.

Design implications:
- Emit portfolio-level metrics (exposure by segment, vintage performance) as scheduled artifacts.
- Support soft-constraints or throttles triggered by concentration alerts rather than hard global blocks.

---
### 11. Liquidity, Provisioning & Accounting Considerations

Credit decisions interact with liquidity and provisioning requirements: expected loss feeds provisioning models, while draw patterns affect funding needs.

Relevance to éclat: surface expected-loss aggregates and likely draw rates so downstream finance processes can provision appropriately and flag high-liquidity scenarios.

Design implications:
- Provide nightly aggregates of expected loss and likely disbursement schedules as artifacts.
- Tag offers whose funding would materially impact near-term liquidity for human review.

---
### 12. Behavioral Finance & Demand Elasticity

Customers react to pricing, messaging, and process friction. Elasticities differ across segments; risk-based pricing must balance expected margin against conversion and fairness.

Relevance to éclat: combine resonance/urgency signals with price elasticity estimates to select offers that maximize expected utility while respecting fairness constraints.

Design implications:
- Run A/B experiments for pricing and messaging and store experiment artifacts for causal analysis.
- Avoid discriminatory pricing: monitor outcomes for protected groups and apply conservative constraints when disparities appear.

---
### 13. Model Validation, Backtesting & Practical Checks

Practical validation steps: unit tests for deterministic components, out-of-time backtests for statistical models, post-deployment monitoring (data drift, prediction distribution), and manual spot-checks for edge cases.

Relevance to éclat: embed lightweight validation jobs in CI that run on example artifacts and surface failing checks to the Governance dashboard.

Design implications:
- Keep a catalog of canonical test cases (example leads, edge inputs) that must pass after any model change.
- Automate drift detection and sanity checks on feature distributions.